In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [ ]:
df.head()

In [ ]:
# one reivew
df['review'][0]

# Text cleaning

1. Sample 10000 rows
2. Remove html tags
3. Remove special characters
4. Converting every thing to lower case
5. Removing Stop words (the, an, of...)
6. Stemming

In [ ]:
# 10K sample of data
df = df.sample(10000)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# replacing the sentiment Positive - 1 and Negtive - 0
df['sentiment'].replace({'positive': 1, 'negative': 0}, inplace=True)

In [ ]:
df.head(2)

In [ ]:
# Function to clean html tag
import re
clean = re.compile('<.*?>')
re.sub(clean, '', df.iloc[1].review)

In [ ]:
# function to clean html tag
def clean_html(text):
    clean = re.compile('<.*?>')
    return re.sub(clean,'', text)

In [ ]:
df['review'] = df['review'].apply(clean_html)

In [ ]:
# converting everything to lower case
df['review'] = df['review'].apply(lambda x: x.lower())

In [ ]:
df['review']

In [ ]:
# Function to remove special characters
def remove_special(text):
    x = ''
    
    for i in text:
        if i.isalnum():  # alpha numeric
            x += i
        else:
            x += ' '
    return x

In [ ]:
remove_special(' the classic 18 = or 18+  use of word.it & is calles ')

In [ ]:
df['review'] = df['review'].apply(remove_special)

In [ ]:
# Remove the stop words
import nltk

In [ ]:
from nltk.corpus import stopwords

In [ ]:
# for example

len(stopwords.words('english'))

In [ ]:
def remove_stopwords(text):
    x=[]
    for i in text.split():
        if i not in stopwords.words('english'):
            x.append(i)
            
    y=x[:]
    x.clear()
    return y

In [ ]:
remove_stopwords('Abhishek is a very good boy')

In [ ]:
df['review'] = df['review'].apply(remove_stopwords)

In [ ]:
df.head()

In [ ]:
# perform steming
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
y = []
def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z=y[:]
    y.clear()
    return z

In [ ]:
# how stem works
stem_words(['I', 'loved', 'loving', 'it'])

In [ ]:
df['review'] = df['review'].apply(stem_words)

In [ ]:
df

In [ ]:
# join back
df['review'] = df['review'].apply(lambda x: ' '.join(x))

In [ ]:
df['review']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1000)

In [ ]:
X = cv.fit_transform(df['review']).toarray()

In [ ]:
X.shape

In [ ]:
X

In [ ]:
X[0].mean()

In [ ]:
y = df.iloc[:,-1].values

In [ ]:
y.shape

In [ ]:
# Now we split the data in traning and test
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# Model training 
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

In [ ]:
# Craeting object
clf1 = GaussianNB()
clf2 = MultinomialNB()
clf3 = BernoulliNB()

In [ ]:
clf1.fit(X_train, y_train)
clf2.fit(X_train, y_train)
clf3.fit(X_train, y_train)

In [ ]:
y_pred1 = clf1.predict(X_test)
y_pred2 = clf2.predict(X_test)
y_pred3 = clf3.predict(X_test)

In [ ]:
y_test.shape, y_pred1.shape

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print('Gaussian: ', accuracy_score(y_test, y_pred1))
print('Multinomial: ', accuracy_score(y_test, y_pred2))
print('Bernoulli: ', accuracy_score(y_test, y_pred3))